# 


# ERA5-Land Magdalena River Basin Daily: Dynamical system properties

In this document we will calculate the clasical errors between ERA5 data and local rainfall gauge stations using standard metrics

## 0. Import Libraries and define functions

In [1]:
import pandas as pd 
import numpy as np
import netCDF4 as nc
from math import sqrt
import sklearn.metrics # Machine Learning in Python
import sklearn.neighbors # Machine Learning in Python
import datetime as dt
import scipy
import math
import warnings # Ignore not important warnings
from numba import jit, cuda # Use GPU
import time
from toolz import curry # List processing tools and functional utilities
from operator import sub # Standard operators as functions
import nolds # Nonlinear measures for dynamical systems
from nolitsa import dimension, delay
warnings.filterwarnings("ignore")

In [2]:
@jit(target_backend ="cuda")
def delay_own2(x):
    n = len(x)
    y = x - np.mean(x)  # Center the data
    # Pre-allocate memory for better performance (avoids redundant allocations)
    r = np.zeros(n)
    auto = np.zeros(n)
    # Calculate correlations using a loop for potential CUDA kernel compatibility
    for k in range(n):
        r[k] = np.dot(y[:n - k], y[-(n - k):])  # Efficient dot product for correlation
    # Calculate autocorrelation
    auto[:] = r / (np.var(x) * (np.arange(n, 0, -1)))
    # Find the first delay where the autocorrelation changes sign twice
    da = 0
    while da < n - 2 and auto[da] * auto[da + 1] > 0:
        da += 1
    return da + 1  

In [3]:
def delay_own(x):
    # Returns the optimal Time-Delay from a time series
    # Use the autocorrelation function and the mutual information score
    da=0;dm=0
    n = len(x)
    y = x-x.mean()   
    r = np.correlate(y, y, mode = 'full')[-n:]
    assert np.allclose(r, np.array([(y[:n-k]*y[-(n-k):]).sum() for k in range(n)]))
    auto = r/(x.var()*(np.arange(n, 0, -1)))
    while (auto[da]*auto[da+1])>0:
        da=da+1
    da=da+1    
    #while sklearn.metrics.mutual_info_score(None,None,contingency=np.histogram2d(x, np.roll(x,dm),20)[0])>=sklearn.metrics.mutual_info_score(None,None, contingency=np.histogram2d(x, np.roll(x,dm+1),20)[0]):
    #    dm=dm+1
    #lag=min(da,dm)+1
    return da #lag       

def delay_tot(x):
    # Returns the optimal Time-Delay from a time series
    # Use the autocorrelation function and the mutual information score
    da=0;dm=0
    n = len(x)
    y = x-x.mean()   
    r = np.correlate(y, y, mode = 'full')[-n:]
    assert np.allclose(r, np.array([(y[:n-k]*y[-(n-k):]).sum() for k in range(n)]))
    auto = r/(x.var()*(np.arange(n, 0, -1)))
    while (auto[da]*auto[da+1])>0:
        da=da+1
    da=da+1    
    while sklearn.metrics.mutual_info_score(None,None,contingency=np.histogram2d(x, np.roll(x,dm),20)[0])>=sklearn.metrics.mutual_info_score(None,None, contingency=np.histogram2d(x, np.roll(x,dm+1),20)[0]):
        dm=dm+1
    lag=min(da,dm)
    return lag       

def global_false_nearest_neighbors(x, lag, min_dims=1, max_dims=15, **cutoffs):
    # Created by hsharrison
    # pypsr taken from https://github.com/hsharrison/pypsr MIT License
    x = _vector(x)
    dimensions = np.arange(min_dims, max_dims + 1)
    false_neighbor_pcts = np.array([_gfnn(x, lag, n_dims, **cutoffs) for n_dims in dimensions])
    return dimensions, false_neighbor_pcts

def _gfnn(x, lag, n_dims, **cutoffs):
    # Created by hsharrison
    # pypsr taken from https://github.com/hsharrison/pypsr MIT License
    # Global false nearest neighbors at a particular dimension.
    # Returns percent of all nearest neighbors that are still neighbors when the next dimension is unfolded.
    # Neighbors that can't be embedded due to lack of data are not counted in the denominator.
    offset = lag*n_dims
    is_true_neighbor = _is_true_neighbor(x, _radius(x), offset)
    return np.mean([
        not is_true_neighbor(indices, distance, **cutoffs)
        for indices, distance in _nearest_neighbors(reconstruct(x, lag, n_dims))
        if (indices + offset < x.size).all()
    ])
      
@curry 
def _is_true_neighbor(x, attractor_radius, offset, indices, distance,relative_distance_cutoff=15, relative_radius_cutoff=2):
    # Created by hsharrison
    # pypsr taken from https://github.com/hsharrison/pypsr MIT License
    distance_increase = np.abs(sub(*x[indices + offset])) 
    return (distance_increase / distance < relative_distance_cutoff and
            distance_increase / attractor_radius < relative_radius_cutoff)
 
@jit 
def _distance(y):
    distances, indices = sklearn.neighbors.NearestNeighbors(n_neighbors=2, algorithm='kd_tree').fit(y).kneighbors(y)
    return distances, indices

def _nearest_neighbors(y):
    # Created by hsharrison
    # pypsr taken from https://github.com/hsharrison/pypsr MIT License
    distances, indices = _distance(y)
    for distance, index in zip(distances, indices):
        yield index, distance[1]

@jit 
def _radius(x):
    # Created by hsharrison
    # pypsr taken from https://github.com/hsharrison/pypsr MIT License
    return np.sqrt(((x - x.mean())**2).mean())

def reconstruct(x, lag, n_dims):
    # create the delayed vector from a time series
    x = _vector(x)
    lags = lag * np.arange(n_dims)
    return np.vstack(x[lag:lag - lags[-1] or None] for lag in lags).transpose()

def deconstruct(x, lag, n_dims):
    # create the time series from a delayed vector
    dec=np.empty(len(x)+lag*(n_dims-1))* np.nan
    dec[:len(x)]=x[:,0]
    dec[len(x):]=x[-lag*(n_dims-1):,-1]
    return dec

def _vector(x):
    # Created by hsharrison
    # pypsr taken from https://github.com/hsharrison/pypsr MIT License
    x = np.squeeze(x)
    if x.ndim != 1:
        raise ValueError('x(t) must be a 1-dimensional signal')
    return x   

def dynamics_analysis(x):   
    # Chaos analyis of the time series
    # Use Eckmann et al algorithm for lyapunov exponents and FNN for embedding dimension
    # Returns the time delay, the embedding dimension and the lyapunov spectrum
    #
    x=x.flatten()
    lag=delay_tot(x)
    mmax=2*int(np.floor(2*math.log10(len(x))))+1 
    fnn=global_false_nearest_neighbors(x, lag, min_dims=1, max_dims=mmax)  
    f_th=0.20
    if len(fnn[1][fnn[1]<=f_th])!=0: 
        if 0 in fnn[1]:
            m1=np.where(fnn[1]==0)[0][0]
        else:
            m1=np.where(fnn[1]<=f_th)[0][0]  
        lyapunov=nolds.lyap_e(x,emb_dim=m1,matrix_dim=m1,tau=lag)
    else:
        m1=-99
        lyapunov=[-99]
    hurst=nolds.hurst_rs(x)
    return lag,m1,lyapunov,hurst

## 1. Load Data

#### **Local Data CSV**

In [4]:
# CSV Paths
data_path="Data/Local CSV/Precipitation 1980-2020 data filled 1.csv" 
coords_path="Data/Local CSV/Precipitation 1980-2020 coord filled.csv" 
# Import data
stations=pd.read_csv(data_path,sep=',',index_col="Fecha",parse_dates=True)
coords=pd.read_csv(coords_path,sep=';',index_col="Station")
# Longitude Coords
if  (coords['Long']<0).any():
    coords['Long']=coords['Long']+360   
# Dates obtained from CSV
dates=stations.index 

#### **ERA-Land Data**

In [5]:
# NC path
eraland_path="Data/ERALand NC/precipitation_eraland_d2.nc"
# Check NetCDF Time
eralandnc = nc.Dataset(eraland_path)
time_unit=eralandnc.variables['time'].units 
time_cal=eralandnc.variables['time'].calendar
time_valh=eralandnc.variables['time'][:]
time_his=nc.num2date(time_valh,units=time_unit,calendar=time_cal)
time_his=sorted([dt.datetime.strptime(k.strftime('%Y-%m-%d %H:%M'),'%Y-%m-%d %H:%M') for k in time_his])
# NetCDF grid data
nc_lat_eraland=eralandnc.variables['latitude'][:]
nc_lon_eraland=eralandnc.variables['longitude'][:]
if  (nc_lon_eraland<0).any():
    nc_lon_eraland=nc_lon_eraland+360   
# Precipitation variable name
var_type4=list(eralandnc.variables.keys())[-1]    
# Load data
eraland=eralandnc.variables[var_type4][:,:,:].data
eralandnc.close()

#### **Basin Mask 0.10**

In [6]:
### Basin Mask 0.10 (.nc file)
rain_nc="Data/Magdalena/GIS/Basin/Basin_ext_010.nc"
basin = nc.Dataset(rain_nc)
basin_lat_010=basin.variables['lat'][:]
basin_lon_010=basin.variables['lon'][:]
var_type=list(basin.variables.keys())[0]  
cells_x_010=np.array(basin_lat_010)[:]
cells_y_010=np.array(basin_lon_010)[:]
mascara=np.array(basin.variables[var_type][:,:])
mask=mascara!=mascara[1,1]
mask=np.flip(mask*1,axis=0).astype('float64')
mask[mask==0]=np.nan
mask_st=mask.reshape((1, mask.shape[0],mask.shape[1]))
mask_basin_010=mask_st.reshape(mask_st.shape[1],mask_st.shape[2])
basin.close()
# Find coordinates where basin exists in mask
basin_coords_010=np.where(mask_basin_010==1)
# Find position of coordinates in datasets arrays
basin_true_longs_010=basin_lon_010.data[basin_coords_010[1]]+360
basin_true_lats_010=np.flip(basin_lat_010.data)[basin_coords_010[0]]

## 2. Dynamical system properties

#### **Local Gauge Stations**

In [ ]:
# Pre-allocate results
sta_results=coords.copy().drop('Length',axis=1)
sta_results["delay"]=np.nan
sta_results["dim_fnn"]=np.nan
sta_results["hurst"]=np.nan
sta_results["lyapunov_max"]=np.nan
sta_results["lyapunov_sum"]=np.nan
stations_names= coords.index
# Compute dynamical system properties
print(len(stations_names))
for ii in range(len(stations_names)):
    print(ii)
    i=stations_names[ii]
    x=stations[i].values
    lag,m1,lyapunov,hurst=dynamics_analysis(x)
    sta_results["delay"].loc[i]=lag
    sta_results["dim_fnn"].loc[i]=m1
    sta_results["hurst"].loc[i]=hurst
    sta_results["lyapunov_max"].loc[i]=max(lyapunov)
    sta_results["lyapunov_sum"].loc[i]=sum(lyapunov)
sta_results.to_csv("Results/Dynamical system properties/local_stations_daily.csv")

#### **ERA-Land Data**

In [ ]:
##### Compute properties ######
# Pre-allocate results
nc_array=np.zeros((eraland.shape[1],eraland.shape[2]))*np.nan
delay_array=nc_array.copy()
dim_fnn_array=nc_array.copy()
dim_afn_array=nc_array.copy()
hurst_array=nc_array.copy()
dynamics_afn_array=nc_array.copy()
dynamics_lyap_array=nc_array.copy()
dynamics_hurst_array=nc_array.copy()
lyapunov_ma_array=nc_array.copy()
lyapunov_sum_array=nc_array.copy()
# Iterate over basin pixels
for j in range(len(basin_coords_010[0])):
    print(str(j)+' of '+str(len(basin_coords_010[0])))
    # Array position 
    x_long=basin_coords_010[0][j]
    y_lat=basin_coords_010[1][j]
    # Geographic position
    look_long=basin_true_longs_010[j]
    look_lat=basin_true_lats_010[j]
   # ERA5 position:
    ## Lat
    lat_eraland_pos=np.where((nc_lat_eraland==look_lat))
    if len(lat_eraland_pos[0])==0:
        continue
    lat_eraland=lat_eraland_pos[0]
    ## Long
    long_eraland_pos=np.where((nc_lon_eraland==look_long))
    if len(long_eraland_pos[0])==0:
        continue
    long_eraland=long_eraland_pos[0]
    eraland_value=eraland[:,lat_eraland,long_eraland].flatten()
    if int(np.mean(eraland_value))==-32767000:
        print('  not available')
        continue
    ## Calculate
    lag,m1,m2,lyapunov,hurst=dynamics_analysis(eraland_value)
    ## Store in array
    delay_array[lat_eraland,long_eraland]=lag
    dim_fnn_array[lat_eraland,long_eraland]=m1
    hurst_array[lat_eraland,long_eraland]=hurst
    lyapunov_ma_array[lat_eraland,long_eraland]=max(lyapunov)
    lyapunov_sum_array[lat_eraland,long_eraland]=sum(lyapunov)

In [ ]:
##### SAVE results TO NC FILE ######
# Define Path 
ncfile = nc.Dataset("Results/Dynamical system properties/Eraland_daily.nc",mode='w',format='NETCDF3_64BIT_OFFSET') 
# Creating netcdf output and its dimensions
lat_dim = ncfile.createDimension('latitude',eraland.shape[1])  
lon_dim = ncfile.createDimension('longitude',eraland.shape[2]) 
ncfile.title="eraland"
lat = ncfile.createVariable('latitude', np.float32, ('latitude',))
lat.units = idw10nc.variables['latitude'].units
lat[:] = nc_lat_eraland
lon = ncfile.createVariable('longitude', np.float32, ('longitude',))
lon.units = idw10nc.variables['longitude'].units
lon[:] = nc_lon_eraland
# Define properties
properties=['delay','dim_fnn','hurst','lyapunov_max','lyapunov_sum']
properties_arrays=[delay_array,dim_fnn_array,hurst_array,lyapunov_ma_array,lyapunov_sum_array]
# Compute properties
for i in range(len(properties)):
    var_nc= ncfile.createVariable(properties[i],np.float64,('latitude','longitude')) 
    var_nc.missing_value=-9999   
    var_nc[:,:] = properties_arrays[i]
ncfile.close()